In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import math
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline
#from datetime import date

In [4]:
#BANKRUPTCY DATASET
bankruptcy = pd.read_csv("D:/research-assistant/datasets/bankruptcy.csv")
bankruptcy.columns = bankruptcy.columns.str.lower() # lower casing all values
bankruptcy.rename(columns={'company_fkey':'cik'},inplace=True)   # renaming column for similarity
bankruptcy = bankruptcy[['id', 'cik','bank_begin_date']] # only considering these columns in data

In [5]:
#COMPANY DATASET
company = pd.read_csv('D:/research-assistant/datasets/comp.csv')
company.columns = company.columns.str.lower() # lower casing all values
# only considering these columns in data
company = company[['cik','datadate','year','size','ni','zscore','leverage','prloss','chgdt','cfo','liquidity','mkbk','litigate','reportlag','priorgc','newequity','newdebt','dependence','indexpert','tenure','big4']]

In [6]:
# checking shapes oof data
print(str('bankruptcy dataset size:'), bankruptcy.shape)
print(str('company dataset size:'), company.shape)

bankruptcy dataset size: (2746, 3)
company dataset size: (131890, 21)


In [7]:
# converting the Bank date to datetime format for easy preprocessing
for i in tqdm(range(bankruptcy.shape[0])):
    obj = bankruptcy['bank_begin_date'].iloc[i]
    date_time_obj = datetime.strptime(obj, '%m/%d/%Y')
    bankruptcy['bank_begin_date'].iloc[i] = date_time_obj                                       

100%|████████████████████████████████████████████████████████████████████████████| 2746/2746 [00:00<00:00, 3293.48it/s]


In [8]:
# converting the Data date to datetime format for easy preprocessing
for i in tqdm(range(company.shape[0])):
    obj = company['datadate'].iloc[i]
    date_time_obj = datetime.strptime(obj, '%m/%d/%Y')
    company['datadate'].iloc[i] = date_time_obj                                       

100%|█████████████████████████████████████████████████████████████████████████| 131890/131890 [10:31<00:00, 208.85it/s]


In [11]:
# creating a list of the ID's of rows which contain are essentially data on a company going bankrupt the 2nd time
unrequired = []
for i in tqdm(range(bankruptcy.shape[0])):
    key1, date1 = bankruptcy[['cik','bank_begin_date']].iloc[i]
    #print(str('K1')+' '+ str(key1))
    j = i+1
    try:
        key2, date2 = bankruptcy[['cik','bank_begin_date']].iloc[j]
        #print(str('K2')+' ' + str(key2))
        if (key1 == key2):
            if ( date2 > date1):
            #print(str('id')+ ' ' + str(bankruptcy['id'].iloc[j]))
                unrequired.append(bankruptcy['id'].iloc[j])
            else:
                None
            #print('----')
        else:
            None
            #print('----')
   
    except:
        None
    

100%|█████████████████████████████████████████████████████████████████████████████| 2746/2746 [00:03<00:00, 733.10it/s]


In [12]:
# number of companies which have gone bankrupt more than once
unrequired = pd.DataFrame(unrequired, columns = ['number'])
len(unrequired)

53

In [13]:
## rechecking above classification
bankruptcy['que'] = bankruptcy['id'].isin(unrequired['number'])
bankruptcy.que.sum()

53

In [14]:
# updating bankruptcy dataset all taking frst banruptcy values
bankruptcy_main = bankruptcy[bankruptcy.que == False]
bankruptcy_main = bankruptcy_main.drop_duplicates(subset = ['cik','bank_begin_date'], keep = 'first') ## dropping duplicates
bankruptcy_main.shape

(2691, 4)

In [15]:
# Checking number of companies in compnay dataset in bankruptcy dataset
company['bankruptcy'] = company['cik'].isin(bankruptcy_main['cik'])
company['bankruptcy'] = company['bankruptcy'].fillna(0)
company = company.set_index('cik')
company['bankruptcy'].sum()

9479

In [16]:
# Merging datasets
combo = pd.merge(company,
                 bankruptcy_main[['cik', 'bank_begin_date']],
                 on='cik', 
                 how='outer', 
                 indicator=True)

In [17]:
combo.columns

Index(['cik', 'datadate', 'year', 'size', 'ni', 'zscore', 'leverage', 'prloss',
       'chgdt', 'cfo', 'liquidity', 'mkbk', 'litigate', 'reportlag', 'priorgc',
       'newequity', 'newdebt', 'dependence', 'indexpert', 'tenure', 'big4',
       'bankruptcy', 'bank_begin_date', '_merge'],
      dtype='object')

In [18]:
# removing redundant data
combo= combo[combo._merge != 'right_only']

In [19]:
combo.shape # checking shape

(131890, 24)

In [20]:
combo.cik.nunique() # checking unique companies

16726

In [21]:
combo.bankruptcy.sum() # checking bankrupt companies

9479

In [22]:
# converting date into required format
combo['datadate'] = pd.to_datetime(combo['datadate'])
combo['bank_begin_date'] = pd.to_datetime(combo['bank_begin_date'], errors='coerce')

In [23]:
# Calculating difference in date 
diff=[]
for i in tqdm(range(combo.shape[0])):
    time = combo.bank_begin_date.iloc[i] - combo.datadate.iloc[i]
    diff.append(time.days) 


100%|███████████████████████████████████████████████████████████████████████| 131890/131890 [00:03<00:00, 35760.38it/s]


In [24]:
# removing nan values in diff
diff = [0 if math.isnan(x) else x for x in diff]

In [25]:
# Appending column diff 
combo['datadiff'] = diff
#merged['datadiff'].fillna(0, inplace = True)
combo['datadiff'].value_counts()

 0       122414
 457         11
 1215        11
 1187        10
 850         10
          ...  
-2999         1
-4918         1
 2633         1
 584          1
-2495         1
Name: datadiff, Length: 5084, dtype: int64

In [26]:
# only condisering one year before bankruptcy data
bank_list =[]
for i in tqdm(range(combo.shape[0])):
    x = combo.datadiff.iloc[i]
    #print(x)
    if(x == 0):
        bank_list.append(2)
    else:
        x = combo.datadiff.iloc[i]
        if x < 366 and x >0:
            bank_list.append(1)
        else:
            bank_list.append(0)

100%|███████████████████████████████████████████████████████████████████████| 131890/131890 [00:01<00:00, 73821.25it/s]


In [27]:
bank_list = pd.DataFrame(bank_list, columns =['type'])
bank_list.type.value_counts()

2    122414
0      8614
1       862
Name: type, dtype: int64

In [28]:
# applying above requirement
combo['bank_list'] = bank_list

In [29]:
# creating new updated dataset
combo_new = combo[combo.bank_list != 0]

In [30]:
combo_new.shape

(123276, 26)

In [31]:
combo_new.cik.nunique()

16012

In [32]:
combo_new.bank_list.value_counts()

2    122414
1       862
Name: bank_list, dtype: int64

In [33]:
combo_new = combo_new.drop(columns = ['_merge'])

In [34]:
combo_new.columns

Index(['cik', 'datadate', 'year', 'size', 'ni', 'zscore', 'leverage', 'prloss',
       'chgdt', 'cfo', 'liquidity', 'mkbk', 'litigate', 'reportlag', 'priorgc',
       'newequity', 'newdebt', 'dependence', 'indexpert', 'tenure', 'big4',
       'bankruptcy', 'bank_begin_date', 'datadiff', 'bank_list'],
      dtype='object')

In [35]:
combo_new = combo_new.sort_values(['cik', 'datadate'], ascending=[True, True])

In [36]:
#dropping duplicates
combo_newest = combo_new.drop_duplicates( subset = ['cik'], keep = 'first')

In [37]:
combo_newest.shape

(16012, 25)

In [38]:
# updating new dataset
combo1 = combo_new[combo_new.bank_list == 1]

In [39]:
combo1.cik.value_counts()

1052163    2
1494448    2
790526     1
1002125    1
744106     1
          ..
1078577    1
889237     1
109870     1
1097002    1
831489     1
Name: cik, Length: 860, dtype: int64

In [40]:
# removing noise if cfo < 0 & ni < 0
noise = []
for i in tqdm(range(combo_newest.shape[0])):
    cfo = combo_newest.cfo.iloc[i]
    ni = combo_newest.ni.iloc[i]
    if(cfo<0 or ni<0):
        None
    else:
        noise.append(combo_newest.cik.iloc[i])

100%|█████████████████████████████████████████████████████████████████████████| 16012/16012 [00:00<00:00, 36069.73it/s]


In [41]:
len(noise)

8530

In [42]:
combo_newest.shape

(16012, 25)

In [43]:
combo_newest.bankruptcy.sum()

860

In [44]:
final = combo_newest.drop(combo_newest[combo_newest.cik.isin(noise)].index.tolist())

In [45]:
final.columns

Index(['cik', 'datadate', 'year', 'size', 'ni', 'zscore', 'leverage', 'prloss',
       'chgdt', 'cfo', 'liquidity', 'mkbk', 'litigate', 'reportlag', 'priorgc',
       'newequity', 'newdebt', 'dependence', 'indexpert', 'tenure', 'big4',
       'bankruptcy', 'bank_begin_date', 'datadiff', 'bank_list'],
      dtype='object')

In [46]:
final_data = final.drop(columns =['datadate','datadiff','bank_begin_date','bank_list'])
final_data = final_data.fillna(0)
final.bankruptcy.sum()/final.shape[0] * 100

10.892809409248864

In [55]:
final_data.to_csv('D:/research-assistant/datasets/final.csv')